# Algoritmia
## Práctica Obligatoria 3
### Curso 2023 - 2024
###### Programación dinámica
---
 

#### Autores:
* César Rodríguez Villagrá

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar librerías nativas (https://docs.python.org/es/3.8/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [14]:
#testeable
# Imports útiles

In [15]:
#testeable
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """

    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size

        self.espectadores: dict = {}

    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """
        return hash((self.name, self.size))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"{self.name}"

    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """
        return str(self)

    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """
        self.espectadores[country] = users

    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """
        return self.espectadores.get(country, 0)

    def tiempo_emision(self, country):
        """Dado un país, obtiene el tiempo de emisión.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de minutos de emisión para el país `country`
        """
        return self.get_users(country)*self.size

In [16]:
#testeable

class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """

    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity

        self.vidAlm: set[Video] = set()

    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """
        return hash((self.identifier, self.country, self.capacity))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"Servidor {self.identifier} en {self.country} con capacidad de {self.capacity} GB"

    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """
        return f"{self.identifier}-{self.country}-{self.capacity}GB"

    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar aquellos que se van a almacenar en el servidor.
           Se ha de optimizar para que el tiempo de emisión sea el máximo posible.
           No se pueden partir los vídeos.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """

        # Para que pasen correctamente los test
        videos = list(reversed(videos))
        # Se puede eliminar la línea anterior, no afecta al algoritmo de la mochila

        tabla: list[list[int]] = []
        tabla.append([0]*(self.capacity+1))

        for i, video in enumerate(videos):
            i += 1
            tabla.append([])
            for j in range(self.capacity+1):
                if j-video.size < 0:
                    tabla[i].append(tabla[i-1][j])
                else:
                    anade: int = tabla[i-1][j-video.size] + \
                        video.tiempo_emision(self.country)
                    ant: int = tabla[i-1][j]
                    if anade >= ant:
                        tabla[i].append(int(anade))
                    else:
                        tabla[i].append(int(ant))

        j: int = self.capacity
        lista = list(enumerate(videos, 1))
        for i, vid in lista[::-1]:
            if tabla[i][j] != tabla[i-1][j]:
                j = j-vid.size
                self.vidAlm.add(vid)
            if j <= 0:
                break

    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión óptimo del servidor.

        Returns
        -------
        number
            Tiempo de Emision            
        """
        tEmision = 0.0
        for video in self.vidAlm:
            tEmision += video.tiempo_emision(self.country)
        return tEmision

    def almacenados(self):
        """A partir de los datos almacenados
           devolver los objetos vídeo.

        Returns
        -------
        collection
            Colección de videos almacenados en el servidor.
        """
        return self.vidAlm

In [17]:
#testeable

class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """

    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """
        self.servidores = set(servidores)
        self.distancias = distancias

        self.minCamino: dict[ServidorCache,
                             dict[ServidorCache, ServidorCache]] = {}
        self.minDist: dict[ServidorCache,
                           dict[ServidorCache, int]] = distancias.copy()

    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """
        return self.distancias

    def calcula_distancias(self):
        """Calcula las distancias MÍNIMAS entre servidores cache
           y los correspondientes caminos.
        """

        for servidor in self.servidores:
            self.minCamino[servidor] = {}

        for sv1 in self.servidores:
            for sv2 in self.servidores:
                for sv3 in self.servidores:
                    if self.minDist[sv2][sv3] > self.minDist[sv2][sv1] + self.minDist[sv1][sv3]:
                        self.minDist[sv2][sv3] = self.minDist[sv2][sv1] + \
                            self.minDist[sv1][sv3]
                        self.minCamino[sv2][sv3] = sv1

    def distancia(self, origen, destino):
        """
        Devuelve la distancia mmochila[j], left + videos[i].tiempo_emision(self.country)ínima entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        int
            Distancia mínima en milisegundos entre los servidores.
        """
        if origen == destino:
            return 0
        return self.minDist[origen][destino]

    def camino(self, origen, destino):
        """
        Devuelve el camino mínimo entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        list<ServidorCache>
            Lista de servidores para llegar de origen a destino. 
            Se debe incluir al origen y al destino.
        """
        if origen == destino:
            return [origen]

        camino: list[ServidorCache] = []
        izq: list[ServidorCache] = []
        der: list[ServidorCache] = []

        sig: ServidorCache | None = self.minCamino.get(origen).get(destino)

        if sig == None:
            return [origen, destino]

        izq = self.camino(origen, sig)
        der = self.camino(sig, destino)

        for elem in izq:
            camino.append(elem)
        for elem in der[1:]:
            camino.append(elem)
        return camino

### Pruebas de ejemplo

In [18]:
import unittest
import json

def carga_dataset(data):
    with open(data) as f:
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)

    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(s["identifier"], s["country"], s["size"])


    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            ping = pings[s.country][p]
            if pings[s.country][p] == -1:
                ping = float("inf")
            p_[s][servers[p]] = ping
    maestro = ServidorMaestro(servers.values(), p_)

    return videos, servers, maestro

class TestBasico(unittest.TestCase):
    
    def test_carga_simple(self):
        
        v, s, m = carga_dataset("toy_PD.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 55800)
        almacenados = spain.almacenados()
        self.assertEqual(len(almacenados), 3)

        m.calcula_distancias()
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.distancia(s["France"], s["Ireland"]), 200)      
        

In [19]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de la librería `nbformat`

```bash
$ pip install nbformat
```

###### Explicación de los tests
* `test_ej1_tiempo_emision_correcto`: Comprueba que el tiempo de emisión es el máximo para cada servidor.
* `test_ej1_emision_maxima`: Comprueba que ante un servidor virtualmente infinita el tiempo de emisión es el máximo posible.
* `test_ej1_pais_no_existe`: Comprueba que el tiempo de emisión de un país que no existe es 0.
* `test_ej2_distancias_correctas`: Comprueba que la distancia es la mínima posible.
* `test_ej2_caminos_correctos`: Comprueba que los caminos son los mínimos.
* `test_ej2_camino_nulo`: Comprueba que ante un viaje cuyo origen es igual al destino el camino es solo el nodo y el coste es 0.

---

### **Informe**
Contesta a las siguientes preguntas (Justificando las respuestas).

#### **Complejidad**

1. Método `ServidorCache.rellena`
    * **Complejidad temporal**: La complejidad del método rellena que implementa la mochila mediante programación dinámica, que tiene complejidad O(W*n), donde W es la capacidad de la "mochila", en este caso del servidor, y n es el número de elementosos que se quieren guardar.
2. Método `ServidorMaestro.calcula_distancias`
    * **Complejidad temporal**: Tiene una complejidad de O(n^3), ya que tiene que recorrer para cada conexión entre 2 servidores, todos los servidores, para encontrar si hay algún camino que sea menor.


#### **Respecto a la maximización del tiempo de emisión del servidor caché.**

* ¿Cómo de buena es la solución? ¿Es óptima o aproximada?<br><sub style="font-style: italic">No confundir una solución óptima (obtiene el máximo posible) frente a una solución eficiente (calcula el máximo con la menor complejidad posible).</sub>

Es una solución óptima, ya que el algoritmo de la mochila implementado garantiza que el valor final, es el máximo posible, en este caso, garantiza que el tiempo de emisión es el máximo posible con esos elementos.

* ¿Cómo cambiaría la complejidad espacial si aumentase el tamaño del servidor? ¿Y si aumentase el número de vídeos?

Como la complejidad espacial de la implementación es O(W*n) donde W es el tamaño del servidor y n, el número de videos con los que se quieren rellenar el servidor. Si aumentas el tamaño del servidor la complejidad espacial no cambiaría, ya que ya está en función del tamaño del servidor(W).
Y si se aumentase el número de vídeos la complejidad tampoco aumentaría ya que está en funcion de los vídeos (n)

#### **Respecto a la búsqueda de distancias mínimas entre servidores.**

* Este problema tiene una aproximación voraz, ¿cuál sería la ventaja de usarlo frente a la solución que has planteado? 

Que a la hora de devolver el camino, con dikstra se puede devolver el camino calculado, que se haya almacenado, en cambio con Floyd para encontrar el camino tienes que hacer más cálculos, ya que tienes el nodo origen, final y uno intermedio que hay que pasar por él, y hay que calcular el mínimo camino entre el origen e intermecio y el intermecio y final, así recursivamente. Por lo que la función camino en Floyd tiene una complejidad mayor que en Dikstra

* ¿Y cuál sería su desventaja?

Es más difícil de implementar, ya que el Algoritmo de Floyd es muy sencillo de implementar.